In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import unsloth
import peft
import transformers
from typing import cast
import vllm


class CausallLM(transformers.PreTrainedModel, transformers.GenerationMixin): ...

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-29 18:40:07 __init__.py:207] Automatically detected platform cuda.


In [3]:
lora_rank = 32
# model_name = "NousResearch/Hermes-2-Theta-Llama-3-8B"
model_name = "bradhiltonendercorp/avocado-plum-berry"

model, tokenizer = cast(
    tuple[CausallLM, transformers.PreTrainedTokenizerBase],
    unsloth.FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=8192,
        load_in_4bit=True,  # False for LoRA 16bit
        fast_inference=True,  # Enable vLLM fast inference
        # vLLM args
        disable_log_requests=True,
        disable_log_stats=False,
        enable_prefix_caching=True,
        gpu_memory_utilization=0.62,  # Reduce if out of memory
        max_lora_rank=lora_rank,
        # max_num_seqs=1024,
        # enforce_eager=True,
        num_scheduler_steps=16,
        use_async=True,
    ),
)
vllm_engine = cast(vllm.AsyncLLMEngine, model.vllm_engine)
peft_model = cast(
    peft.peft_model.PeftModelForCausalLM,
    unsloth.FastLanguageModel.get_peft_model(
        model,
        r=lora_rank,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],  # Remove QKVO if out of memory
        lora_alpha=lora_rank,
        # Enable long context finetuning
        use_gradient_checkpointing="unsloth",  # type: ignore
        random_state=3407,
    ),
)
lora_model = cast(peft.tuners.lora.LoraModel, peft_model.base_model)
peft_model

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading NousResearch/Hermes-2-Theta-Llama-3-8B with actual GPU utilization = 61.53%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.11 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 33.56 GB. Also swap space = 6 GB.
INFO 03-29 18:40:14 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config

tokenizer_config.json:   0%|          | 0.00/56.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

...

Capturing CUDA graph shapes: 100%|██████████| 43/43 [00:25<00:00,  1.67it/s]

INFO 03-29 18:41:04 model_runner.py:1562] Graph capturing finished in 26 secs, took 5.54 GiB
INFO 03-29 18:41:04 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 31.26 seconds


adapter_model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128257, 4096, padding_idx=128001)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Li

In [4]:
from vllm.worker.worker_base import WorkerWrapperBase
from vllm.worker.multi_step_model_runner import MultiStepModelRunner

worker = cast(
    WorkerWrapperBase, getattr(vllm_engine.engine.model_executor, "driver_worker")
)
multi_step_model_runner: MultiStepModelRunner = worker.model_runner
base_model_runner = multi_step_model_runner._base_model_runner

In [6]:
vllm_engine.engine.model_executor.cache_config.num_gpu_blocks * vllm_engine.engine.model_executor.cache_config.block_size

341072

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from datasets import Dataset, IterableDataset
from trl import GRPOConfig, GRPOTrainer

config = GRPOConfig(
    per_device_train_batch_size=8,
    hub_model_id="avocado-plum-berry",
    max_steps=(2**31 - 1),
)


def generator():
    while True:
        yield {"prompt": ""}


trainer = GRPOTrainer(
    model=peft_model,
    reward_funcs=[],
    args=config,
    # train_dataset=Dataset.from_list([{"prompt": ""} for _ in range(1_000_000)]),
    train_dataset=IterableDataset.from_generator(generator),
    processing_class=tokenizer,
)

In [9]:
trainer.save_model("../test-test-test")

/home/gcpuser/sky_workdir/.venv/lib/python3.12/site-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [8]:
trainer.push_to_hub(commit_message="update2")

  0%|          | 0/1 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.88k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bradhiltonendercorp/avocado-plum-berry/commit/17335a550b1d3ded2aeab892899705d56be111b6', commit_message='update2', commit_description='', oid='17335a550b1d3ded2aeab892899705d56be111b6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bradhiltonendercorp/avocado-plum-berry', endpoint='https://huggingface.co', repo_type='model', repo_id='bradhiltonendercorp/avocado-plum-berry'), pr_revision=None, pr_num=None)

In [10]:
trainer._save_optimizer_and_scheduler(trainer.args.output_dir)

In [9]:
trainer.create_optimizer_and_scheduler(1_000_000)

In [7]:
import asyncio


async def warmup(request_id: str) -> None:
    async for _ in vllm_engine.generate(
        prompt={
            "prompt_token_ids": [0]
            * (vllm_engine.engine.scheduler_config.max_model_len - 2)
        },
        sampling_params=vllm.SamplingParams(max_tokens=2),
        request_id=request_id,
    ):
        pass


await asyncio.gather(
    *(warmup(f"{i}") for i in range(vllm_engine.engine.scheduler_config.max_num_seqs))
)

INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 0.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 1.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 2.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 3.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 4.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 5.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 6.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 7.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 8.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 9.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 10.


INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 11.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 12.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 13.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 14.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 15.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 16.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 17.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 18.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 19.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 20.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 21.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 22.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 23.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 24.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added request 25.
INFO 03-26 18:26:09 async_llm_engine.py:211] Added requ

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
"55887MiB"

In [ ]:
type(peft_model.base_model)

peft.tuners.lora.model.LoraModel

In [ ]:
from transformers import models

models.llama.LlamaForCausalLM

from typing import Protocol